**Asynchronous Advantage Actor Critic (A3C)**

- Implementation on Survival Gridworld environment
- Runs on TensorFlow version 1.13 ~ 1.15
- Source code modified from https://github.com/dennybritz/reinforcement-learning/tree/master/PolicyGradient/a3c

In [1]:
import numpy as np
import tensorflow as tf
import sys
import os
import itertools
import collections
import time
import unittest
import shutil
import threading
import multiprocessing
import netfunctions as nf
import environment as _env
import cv2
import math

In [2]:
from inspect import getsourcefile
current_path = os.path.dirname(os.path.abspath(getsourcefile(lambda:0)))
import_path = os.path.abspath(os.path.join(current_path, "../.."))

if import_path not in sys.path:
    sys.path.append(import_path)

In [3]:
# Define use of value clipping and the hyperparameter
use_clip = True
clip_val = -0.80

In [4]:
# Define environment with infinite resource
env_inf_resource = False

In [5]:
# Define learning rate for the shared network
learning_rate = 0.00025
#learning_rate = 0.00035

In [6]:
def shared_model(net_x, alias="Net_Model"):
        
    # Build neural network
    network = nf.NetFunctions(alias)
        
    net = network.conv_layer(net_x, num_filters=16, filter_size=4, use_pooling=True)    #convolutional layer 1
    net = network.conv_layer(net, num_filters=32, filter_size=4, use_pooling=True)      #convolutional layer 2
    net = network.flatten(net)                                                          #flatten input
    net = network.dense_layer(net, num_units=64, use_relu=True)                         #dense layer 1
    net = network.dense_layer(net, num_units=128, use_relu=True)                        #dense layer 2
        
    return net

In [7]:
class PolicyEstimator():
    """
    Policy Function approximator. Given a observation, returns probabilities
    over all possible actions.

    Args:
        num_outputs: Size of the action space.
        reuse: If true, an existing shared network will be re-used.
        trainable: If true we add train ops to the network.
            Actor threads that don't update their local models and don't need
            train ops would set this to false.
    """

    def __init__(self, num_outputs, reuse=False, trainable=True):
        self.num_outputs = num_outputs

        # Placeholders for our input
        # Our input are 16 x 16 current state, previous state, location state, and energy state 
        self.states = tf.placeholder(shape=[None, 16, 16, 4], dtype=tf.float32, name="X")
        # The TD target value
        self.targets = tf.placeholder(shape=[None], dtype=tf.float32, name="y")
        # Integer id of which action was selected
        self.actions = tf.placeholder(shape=[None], dtype=tf.int32, name="actions")

        X = self.states
        batch_size = tf.shape(self.states)[0]

        # Graph shared with Value Net
        with tf.variable_scope("shared", reuse=reuse):
            fc1 = shared_model(X, alias="policy_model")

        with tf.variable_scope("policy_net"):
            self.logits = tf.contrib.layers.fully_connected(fc1, num_outputs, activation_fn=None)
            self.probs = tf.nn.softmax(self.logits) + 1e-8

            self.predictions = {
                "logits": self.logits,
                "probs": self.probs
            }

            # We add entropy to the loss to encourage exploration
            self.entropy = -tf.reduce_sum(self.probs * tf.log(self.probs), 1, name="entropy")
            self.entropy_mean = tf.reduce_mean(self.entropy, name="entropy_mean")

            # Get the predictions for the chosen actions only
            gather_indices = tf.range(batch_size) * tf.shape(self.probs)[1] + self.actions
            self.picked_action_probs = tf.gather(tf.reshape(self.probs, [-1]), gather_indices)

            self.losses = - (tf.log(self.picked_action_probs) * self.targets + 0.03 * self.entropy)
            self.loss = tf.reduce_sum(self.losses, name="loss")

            if trainable:
                # self.optimizer = tf.train.AdamOptimizer(1e-4)
                self.optimizer = tf.train.RMSPropOptimizer(learning_rate, 0.99, 0.0, 1e-6)
                self.grads_and_vars = self.optimizer.compute_gradients(self.loss)
                self.grads_and_vars = [[grad, var] for grad, var in self.grads_and_vars if grad is not None]
                self.train_op = self.optimizer.apply_gradients(self.grads_and_vars, 
                                                               global_step=tf.contrib.framework.get_global_step())

    """END OF POLICYESTIMATOR CLASS"""

In [8]:
class ValueEstimator():
    """
    Value Function approximator. Returns a value estimator for a batch of observations.

    Args:
        reuse: If true, an existing shared network will be re-used.
        trainable: If true we add train ops to the network.
            Actor threads that don't update their local models and don't need
            train ops would set this to false.
    """

    def __init__(self, reuse=False, trainable=True):
        # Placeholders for our input
        # Our input are 16 x 16 current state, previous state, location state, and energy state 
        self.states = tf.placeholder(shape=[None, 16, 16, 4], dtype=tf.float32, name="X")
        # The TD target value
        self.targets = tf.placeholder(shape=[None], dtype=tf.float32, name="y")

        X = self.states

        # Graph shared with Value Net
        with tf.variable_scope("shared", reuse=reuse):
            fc1 = shared_model(X, alias="value_model")

        with tf.variable_scope("value_net"):
            self.logits = tf.contrib.layers.fully_connected(inputs=fc1,num_outputs=1, activation_fn=None)
            self.logits = tf.squeeze(self.logits, squeeze_dims=[1], name="logits")

            self.losses = tf.squared_difference(self.logits, self.targets)
            self.loss = tf.reduce_sum(self.losses, name="loss")

            self.predictions = {
                "logits": self.logits
            }

            if trainable:
                # self.optimizer = tf.train.AdamOptimizer(1e-4)
                self.optimizer = tf.train.RMSPropOptimizer(learning_rate, 0.99, 0.0, 1e-6)
                self.grads_and_vars = self.optimizer.compute_gradients(self.loss)
                self.grads_and_vars = [[grad, var] for grad, var in self.grads_and_vars if grad is not None]
                self.train_op = self.optimizer.apply_gradients(self.grads_and_vars, 
                                                               global_step=tf.contrib.framework.get_global_step())
    
    """END OF VALUEESTIMATOR CLASS"""

In [9]:
class LogFile:
    
    def __init__(self, save_dir):
        
        # Initialize save directory folder
        self.save_dir = save_dir
        
        # Initialize save directory
        self.create_save_dir()
        
    def create_save_dir(self):
        
        # Create the save directory if it does not already exist
        if not os.path.exists(self.save_dir):
            os.makedirs(self.save_dir)
            
    def create_file(self, file_name):
    
        # Get directory and filename
        save_filename = self.save_dir + '/ ' + file_name
        
        # Initialize write lines container
        get_lines = []
        
        # Check if file already exists
        try:
            # Using concept manager in read mode
            with open(save_filename, 'r') as rf:
                get_lines = rf.readlines()
                file_exist = True
        # If not then create new file
        except: 
            # Using context manager in write mode to create file
            with open(save_filename, 'w') as wf:
                get_lines.append('Current Global Steps=0\n')
                get_lines.append('Current Optimization=0\n')
                wf.writelines(get_lines)
                file_exist = False
        
        return file_exist
        
    def save_data(self, file_name, data):
        
        # Data structure:
        # data[0] = Global steps
        # data[1] = Optimization number
        # data[2] = Number of success
        # data[3] = Number of attempts
        # data[4] = Episode score
        # data[5] = Rewards mean
        # data[6] = Mean of minimum distance to finish
        
        # Get directory and filename
        save_filename = self.save_dir + '/ ' + file_name
        
        # Initialize read lines container
        get_lines = []
        
        # If file exists
        try:
            # Using concept manager in read mode
            with open(save_filename, 'r') as rf:
                get_lines = rf.readlines()
            
            # Using context manager in write mode
            with open(save_filename, 'w') as wf:
                get_lines[0] = 'Current Global Steps=' + str(int(data[0])) + '\n'
                get_lines[1] = 'Current Optimization=' + str(int(data[1])) + '\n'
                stp_msg = 'Global_Steps=' + str(int(data[0])) + ' '
                opt_msg = 'Optimization=' + str(int(data[1])) + ' '
                suc_msg = 'Success_Rate=' + str(int(data[2])) + '/' + str(int(data[3])) + ' '
                scr_msg = 'Score_Mean=' + str(data[4]) + ' '
                rwd_msg = 'Rewards_Mean=' + str(data[5]) + ' '
                dst_msg = 'Min_Dist_to_Finish_Mean=' + str(data[6]) + ' '
                lgt_msg = 'Episode_Length=' + str(data[7]) + '\n'
                get_msg = stp_msg + opt_msg  + suc_msg + scr_msg + rwd_msg + dst_msg + lgt_msg
                get_lines.append(get_msg)
                wf.writelines(get_lines)
                print('logfile on global steps=' + str(data[0]) + ' saved...')
            
        # If file does not exist yet
        except:
            print('ERROR: File object does not exist')
            
    def get_header_info(self, file_name):
        
        # Get directory and filename
        save_filename = self.save_dir + '/ ' + file_name
        
        # Initialize read lines container
        get_lines = []
        
        # Initialize return variables
        stp = -1
        opt = -1
        
        # If file exists
        try:
            # Using concept manager in read mode
            with open(save_filename, 'r') as rf:
                get_lines = rf.readlines()
                
            stp = get_lines[0].split("=")
            opt = get_lines[1].split("=")
                
        # If file does not exist yet
        except:
            print('ERROR: File object does not exist')
            
        return (int(stp[1]), int(opt[1]))
    
    def get_saved_data(self, file_name):
        
        # Get directory and filename
        save_filename = self.save_dir + '/ ' + file_name
        
        # Initialize read lines container
        get_lines = []
        
        # Initialize return array
        data = [-1]
        
        # If file exists
        try:
            # Using concept manager in read mode
            with open(save_filename, 'r') as rf:
                get_lines = rf.readlines()
                
            # Reinitialize return array
            data = [[0 for c in range(8)] for r in range(len(get_lines) - 2)]
            
            # Loop through data lines
            for i in range(2, len(get_lines)):
                split_line = get_lines[i].split(" ")
                
                # Get inividual data
                stp = split_line[0].split('=')
                opt = split_line[1].split('=')
                div_srt = split_line[2].split('=')
                srt = div_srt[1].split("/")
                scr = split_line[3].split('=')
                rwd = split_line[4].split('=')
                mdf = split_line[5].split('=')
                lgt = split_line[6].split('=')
                
                # Store data in the return array
                data[i - 2][0] = int(stp[1])
                data[i - 2][1] = int(opt[1])
                data[i - 2][2] = int(srt[0])
                data[i - 2][3] = int(srt[1])
                data[i - 2][4] = float(scr[1])
                data[i - 2][5] = float(rwd[1])
                data[i - 2][6] = float(mdf[1])
                data[i - 2][7] = float(lgt[1])
                
        # If file does not exist yet
        except:
            print('ERROR: File object does not exist')
            
        return data
    
        """END OF LOGFILE CLASS"""

In [10]:
Transition = collections.namedtuple("Transition", ["state", "action", "reward", "next_state", "done"])

In [11]:
# Helper functions

def make_copy_params_op(v1_list, v2_list):
    """
    Creates an operation that copies parameters from variable in v1_list to variables in v2_list.
    The ordering of the variables in the lists must be identical.
    """
    v1_list = list(sorted(v1_list, key=lambda v: v.name))
    v2_list = list(sorted(v2_list, key=lambda v: v.name))

    update_ops = []
    for v1, v2 in zip(v1_list, v2_list):
        op = v2.assign(v1)
        update_ops.append(op)

    return update_ops

def make_train_op(local_estimator, global_estimator):
    """
    Creates an op that applies local estimator gradients
    to the global estimator.
    """
    local_grads, _ = zip(*local_estimator.grads_and_vars)
    # Clip gradients
    local_grads, _ = tf.clip_by_global_norm(local_grads, 4.0)
    _, global_vars = zip(*global_estimator.grads_and_vars)
    local_global_grads_and_vars = list(zip(local_grads, global_vars))
    return global_estimator.optimizer.apply_gradients(local_global_grads_and_vars, 
                                                      global_step=tf.contrib.framework.get_global_step())

In [12]:
class Worker(object):
    """
    An A3C worker thread. Runs episodes locally and updates global shared value and policy nets.

    Args:
        name: A unique name for this worker
        env: The Gym environment used by this worker
        policy_net: Instance of the globally shared policy net
        value_net: Instance of the globally shared value net
        global_counter: Iterator that holds the global step
        discount_factor: Reward discount factor
        max_global_steps: If set, stop coordinator when global_counter > max_global_steps
    """
    def __init__(self, name, env, policy_net, value_net, global_counter, discount_factor=0.99, max_global_steps=None):
        self.name = name
        self.discount_factor = discount_factor
        self.max_global_steps = max_global_steps
        self.global_step = tf.contrib.framework.get_global_step()
        self.global_policy_net = policy_net
        self.global_value_net = value_net
        self.global_counter = global_counter
        self.local_counter = itertools.count()
        self.env = env
        self.start_energy = 10.0

        # Create local policy/value nets that are not updated asynchronously
        with tf.variable_scope(name):
            self.policy_net = PolicyEstimator(policy_net.num_outputs)
            self.value_net = ValueEstimator(reuse=True)

        # Op to copy params from global policy/valuenets
        self.copy_params_op = make_copy_params_op(
            tf.contrib.slim.get_variables(scope="global", collection=tf.GraphKeys.TRAINABLE_VARIABLES),
            tf.contrib.slim.get_variables(scope=self.name+'/', collection=tf.GraphKeys.TRAINABLE_VARIABLES))

        self.vnet_train_op = make_train_op(self.value_net, self.global_value_net)
        self.pnet_train_op = make_train_op(self.policy_net, self.global_policy_net)

        self.state = None

    def run(self, sess, coord, t_max):
        with sess.as_default(), sess.graph.as_default():
            # Initial state
            self.state = self.env.reset(initial_energy=self.start_energy)
            try:
                while not coord.should_stop():
                    # Copy Parameters from the global networks
                    sess.run(self.copy_params_op)

                    # Collect some experience
                    transitions, local_t, global_t = self.run_n_steps(t_max, sess)

                    if self.max_global_steps is not None and global_t >= self.max_global_steps:
                        tf.logging.info("Reached global step {}. Stopping.".format(global_t))
                        coord.request_stop()
                        return

                    # Update the global networks
                    self.update(transitions, sess)

            except tf.errors.CancelledError:
                return

    def _policy_net_predict(self, state, sess):
        feed_dict = { self.policy_net.states: [state] }
        preds = sess.run(self.policy_net.predictions, feed_dict)
        return preds["probs"][0]

    def _value_net_predict(self, state, sess):
        feed_dict = { self.value_net.states: [state] }
        preds = sess.run(self.value_net.predictions, feed_dict)
        return preds["logits"][0]

    def run_n_steps(self, n, sess):
        transitions = []
        for _ in range(n):
            # Take a step
            action_probs = self._policy_net_predict(self.state, sess)
            action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
            next_state, path_reward, reward, done = self.env.step(action)

            # Store transition
            transitions.append(Transition(state=self.state, action=action, reward=reward, next_state=next_state, done=done))

            # Increase local and global counters
            local_t = next(self.local_counter)
            global_t = next(self.global_counter)

            if local_t % 50 == 0:
                tf.logging.info("{}: local Step {}, global step {}".format(self.name, local_t, global_t))

            if done:
                self.state = self.env.reset(initial_energy=self.start_energy)
                break
            else:
                self.state = next_state
        return transitions, local_t, global_t

    def update(self, transitions, sess):
        """
        Updates global policy and value networks based on collected experience

        Args:
            transitions: A list of experience transitions
            sess: A Tensorflow session
        """

        # If we episode was not done we bootstrap the value from the last state
        reward = 0.0
        if not transitions[-1].done:
            reward = self._value_net_predict(transitions[-1].next_state, sess)

        # Accumulate minibatch exmaples
        states = []
        policy_targets = []
        value_targets = []
        actions = []

        for transition in transitions[::-1]:
            if use_clip: reward = max(clip_val, reward) # Value clipping
            reward = transition.reward + self.discount_factor * reward
            policy_target = (reward - self._value_net_predict(transition.state, sess))
            # Accumulate updates
            states.append(transition.state)
            actions.append(transition.action)
            policy_targets.append(policy_target)
            value_targets.append(reward)

        feed_dict = {
            self.policy_net.states: states,
            self.policy_net.targets: policy_targets,
            self.policy_net.actions: actions,
            self.value_net.states: states,
            self.value_net.targets: value_targets,
        }

        # Train the global estimators using local gradients
        global_step, pnet_loss, vnet_loss, _, _ = sess.run([
            self.global_step,
            self.policy_net.loss,
            self.value_net.loss,
            self.pnet_train_op,
            self.vnet_train_op], feed_dict)

        return pnet_loss, vnet_loss
    
    """END OF WORKER CLASS"""

In [13]:
class PolicyMonitor(object):
    """
    Helps evaluating a policy by running an episode in an environment,
    saving a video, and plotting summaries to Tensorboard.
aa
    Args:
        env: Environment to run in
        policy_net: A policy estimator
        opt_counter: Model optimization counter for saver and logging
        global_counter: Global step counter for logging
        saver: Policy model checkpoint saver
        log_dir: Directory for saving test logs
    """
    def __init__(self, env, policy_net, opt_counter, global_counter, saver, log_dir=None):

        self.env = env
        self.start_energy = 10.0
        self.global_policy_net = policy_net
        self.saver = saver
        self.opt_counter = opt_counter
        self.global_counter = global_counter
        self.save_model = False
        self.start_global_t = int(repr(self.global_counter)[6:-1])
        
        self.checkpoint_path = None
        if log_dir != None:
            self.checkpoint_path = os.path.join(log_dir, "model/")

        # Local policy net
        with tf.variable_scope("policy_eval"):
            self.policy_net = PolicyEstimator(policy_net.num_outputs)

        # Op to copy params from global policy/value net parameters
        self.copy_params_op = make_copy_params_op(
            tf.contrib.slim.get_variables(scope="global", collection=tf.GraphKeys.TRAINABLE_VARIABLES),
            tf.contrib.slim.get_variables(scope="policy_eval", collection=tf.GraphKeys.TRAINABLE_VARIABLES))
        
        # Initialize test data logfile
        self.logfile = None
        self.log_filename = 'logfile'
        if log_dir != None:
            save_dir = os.path.join(log_dir, "test_log/")
            self.logfile = LogFile(save_dir)
            check_exist = self.logfile.create_file(self.log_filename)

    def _policy_net_predict(self, state, sess):
        feed_dict = { self.policy_net.states: [state] }
        preds = sess.run(self.policy_net.predictions, feed_dict)
        return preds["probs"][0]

    def eval_once(self, sess, t_epi, render=False):
        with sess.as_default(), sess.graph.as_default():
            # Copy params to local model
            global_step, _ = sess.run([tf.contrib.framework.get_global_step(), self.copy_params_op])
            
            # Save model on condition
            if self.checkpoint_path != None and self.save_model:
                self.saver.save(sess, save_path=self.checkpoint_path, global_step=self.opt_counter)
                print("*****Policy Model Saved Checkpoint...")
                self.opt_counter += 1

            # Run an episode
            done = False
            state = self.env.reset(initial_energy=self.start_energy)
            total_reward = 0.0
            episode_length = 0
            
            # Initialize test statistics
            rewards = 0
            distance = 1000
            score = 0
            success = 0
            length = 0
            
            while not done:
                if render: self.env.render(delay=300)
                action_probs = self._policy_net_predict(state, sess)
                action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
                next_state, path_reward, reward, done = self.env.step(action)
                state = next_state
                
                # Get statistics
                rewards += reward
                length = self.env.step_count
                
                # Get minimum distance to finish
                dsx = math.pow(self.env.endpoint[0] - self.env.control_position[0], 2)
                dsy = math.pow(self.env.endpoint[1] - self.env.control_position[1], 2)
                dst = math.sqrt(dsx + dsy)
                if(dst < distance): distance = int(dst)
                    
                if done:
                    if(self.env.get_success()): success = 1
                        
                    # Get final score
                    score = self.env.get_score()
                        
                    if render:
                        self.env.render(delay=1000)
                        self.env.close_render()

            result_msg = "Episode: {0:3}\t Final Score: {1:5.2f}\t Acc. Reward: {2:5.2f}\t Min_Dist: {3:6.2f}\t Epi_Len: {4:6.2f}"
            print(result_msg.format(t_epi, score, rewards, distance, length))

            return [success, score, rewards, distance, length]
        
    def log(self, stat):
        
        test_log = np.zeros(shape=8, dtype=np.float)
        
        # Log test results
        test_log[0] = float(stat[0])
        test_log[1] = math.floor(self.opt_counter - 1)
        test_log[2] = math.floor(stat[1])
        test_log[3] = math.floor(stat[2])
        test_log[4] = round(stat[3], 2)
        test_log[5] = round(stat[4], 2)
        test_log[6] = round(stat[5], 2)
        test_log[7] = round(stat[6], 2)
        
        # Save test results
        self.logfile.save_data(self.log_filename, test_log)
        
    def run_test(self, sess, num_ep, global_t=0, render=False, log=False):
        """
        Run the policy in test mode.
        """  
        # Initialize total stats
        t_score    = 0
        t_reward   = 0
        t_distance = 0
        t_success  = 0
        t_length   = 0
        for ep in range(num_ep):
            # Set condition for saving the model
            if ep < 1: self.save_model = True
            else: self.save_model = False
            if render: self.save_model = False
                
            stats = self.eval_once(sess, t_epi=ep, render=render)
            t_success  += stats[0]
            t_score    += stats[1]
            t_reward   += stats[2]
            t_distance += stats[3]
            t_length   += stats[4]
            
        # Get average
        ave_score    = t_score / num_ep
        ave_reward   = t_reward / num_ep
        ave_distance = t_distance / num_ep
        ave_length   = t_length / num_ep
        
        if log:
            self.log([global_t, t_success, num_ep, ave_score, ave_reward, ave_distance, ave_length])
        
        test_msg = "Overall Results: Success Rate: {0:2}/{1}, Ave. Score: {2:5.2f}, Ave. Rewards: {3:5.2f}, Ave. Min_Dist {4:5.2f}, Ave. Epi_Length {5:6.2f}"
        print(test_msg.format(t_success, num_ep, ave_score, ave_reward, ave_distance, ave_length))
        
    def continuous_eval(self, eval_every, num_ep, sess, coord):
        """
        Continuously evaluates the policy every [eval_every] global steps.
        """
        try:   
            while not coord.should_stop():
                # Test the policy
                global_t = int(repr(self.global_counter)[6:-1]) - 1
                eval_check = global_t % eval_every
                if eval_check >= 0 and eval_check <= 500 and global_t >= eval_every + self.start_global_t:
                    self.run_test(sess, num_ep, global_t, log=True)
                    
        except tf.errors.CancelledError:
            return

    """END OF POLICYMONITOR CLASS"""

In [14]:
class Train:
    """
    Initializes training variables and requisites

    Args:
            parallelism: Number of threads to run. If not set we run [num_cpu_cores] threads
            max_global_steps: Stop training after this many steps in the environment. Defaults to running indefinitely
            model_dir: Directory to write Tensorboard summaries and videos to
            reset: If set, delete the existing model directory and start training from scratch
            custom_env: Custom environment parameters and description
    """
    def __init__(self, parallelism=None, max_global_steps=None, model_dir="checkpoints/a3c", custom_env=None):
        
        self.max_global_steps = max_global_steps
        self.valid_actions = list(range(4))
        self.discount_factor = 0.98
        
        # Set the number of workers
        self.num_workers = multiprocessing.cpu_count()
        if parallelism != None:
            self.num_workers = parallelism

        # Set save directory
        self.model_dir = model_dir
        self.checkpoint_dir = os.path.join(model_dir, "model/")
        if not os.path.exists(self.checkpoint_dir):
            os.makedirs(self.checkpoint_dir)
        
        # Initialize worker requisites
        self.global_counter = None
        self.global_step = None
        self.policy_net = None
        self.value_net = None
        self.opt_counter = 1
        self.workers = []
        self.saver = None
        self.pe = None
        
        # Initialize workers
        self.init_worker(custom_env)
        
    def make_env(self, env_id, is_default=True, custom_env=None):
        
        env = _env.Environment(env_id=env_id, is_default=is_default, grid_size=[10, 10])
        if custom_env != None:
            env.custom_environment(gridmatrix=custom_env[0], delta_s=custom_env[1], start=custom_env[3], end=custom_env[4])
            env.fstate_size = custom_env[5]
            
        # Set environment with infinite resource
        if(env_inf_resource):
            env.set_inf_resource()
        
        return env
    
    def init_worker(self, custom_env):
        
        with tf.device("/cpu:0"):
            # Keeps track of the number of updates we've performed
            self.global_step = tf.Variable(0, name="global_step", trainable=False)

            # Global policy and value nets
            with tf.variable_scope("global") as vs:
                self.policy_net = PolicyEstimator(num_outputs=len(self.valid_actions))
                self.value_net = ValueEstimator(reuse=True)

            # Global step iterator
            self.global_counter = itertools.count()
            if self.model_dir != None:
                log_dir = os.path.join(self.model_dir, "test_log/")
                if os.path.exists(log_dir):
                    logfile = LogFile(log_dir)
                    stp, opt = logfile.get_header_info('logfile')
                    self.global_counter = itertools.count(start=stp)
                    self.opt_counter = opt + 1

            # Create worker graphs
            for worker_id in range(self.num_workers):
                worker = Worker(name="worker_{}".format(worker_id + 1),
                                env=self.make_env(env_id=worker_id + 1, custom_env=custom_env),
                                policy_net=self.policy_net,
                                value_net=self.value_net,
                                global_counter=self.global_counter,
                                discount_factor = self.discount_factor,
                                max_global_steps=self.max_global_steps)
                if custom_env != None:
                    worker.start_energy = custom_env[2]
                self.workers.append(worker)
                
            self.saver = tf.train.Saver(keep_checkpoint_every_n_hours=5.0, max_to_keep=3)

            # Used to occasionally evaluate the policy and log test results
            self.pe = PolicyMonitor(env=self.make_env(env_id=0, custom_env=custom_env), 
                                    policy_net=self.policy_net, 
                                    opt_counter=self.opt_counter, 
                                    global_counter=self.global_counter, 
                                    saver=self.saver, 
                                    log_dir=self.model_dir)
            if custom_env != None:
                self.pe.start_energy = custom_env[2]
            
    def run(self, t_max=5, eval_every=300, test_epi=30):
        """
        Run training and evaluate the policy

        Args:
                t_max: Number of steps before performing an update
                eval_every: Evaluate the policy every N seconds
        """
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            coord = tf.train.Coordinator()

            # Load a previous checkpoint if it exists
            latest_checkpoint = tf.train.latest_checkpoint(self.checkpoint_dir)
            if latest_checkpoint:
                print("Loading model checkpoint: {}".format(latest_checkpoint))
                self.saver.restore(sess, latest_checkpoint)

            # Start worker threads
            worker_threads = []
            for worker in self.workers:
                worker_fn = lambda worker=worker: worker.run(sess, coord, t_max)
                t = threading.Thread(target=worker_fn)
                t.start()
                worker_threads.append(t)

            # Start a thread for policy eval task
            monitor_thread = threading.Thread(
                             target=lambda: self.pe.continuous_eval(eval_every, test_epi, sess, coord))
            monitor_thread.start()

            # Wait for all workers to finish
            coord.join(worker_threads)
            
    def test(self, num_ep):
        """
        Run the policy in test mode
        
        Args:
            num_ep: Number of episodes to run in render mode
        """
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            
            # Load a previous checkpoint if it exists
            latest_checkpoint = tf.train.latest_checkpoint(self.checkpoint_dir)
            if latest_checkpoint:
                print("Loading model checkpoint: {}".format(latest_checkpoint))
                self.saver.restore(sess, latest_checkpoint)
            
            # Run the policy render mode
            self.pe.run_test(sess, num_ep, render=True)
            
    """END OF TRAIN CLASS"""

# Select Test Environment

In [15]:
_Gridworld = collections.namedtuple("_Gridworld", ["gmatrix", "delta_s", "energy", "start", "end", "fstate"])
def Gridworld(gmatrix, delta_s, energy, start, end, fstate=[5, 5]):
    return _Gridworld(gmatrix, delta_s, energy, start, end, fstate)

In [16]:
# Custom GRIDWORLD A (SPARSE REWARDS TEST)
# Create custom gridworld matrix (9 x 14)
# Optimal Score = 2
gmatrix = [[  0,   0,   0, -10, -20, -20, -20, -20, -20, -20, -20, -10,   0,   0],
           [  0,   0,   0, -10, -10, -10, -10, -10, -10, -10, -10, -10,   0,   0],
           [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
           [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
           [  0,   0,   0,  10,   0,   0,   0,   0,   0,   0,   5,   0,   7,   0],
           [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
           [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
           [  0,   0,   0, -10, -10, -10, -10, -10, -10, -10, -10, -10,   0,   0],
           [  0,   0,   0, -10, -20, -20, -20, -20, -20, -20, -20, -10,   0,   0]]
           
env = Gridworld(gmatrix=gmatrix, delta_s=1.0, energy=10.0, start=[0, 8], end=[13, 8], fstate=[7, 7])
_max_global_steps = 1000000

In [ ]:
# Custom GRIDWORLD B (REWARD TRACING TEST)
# Create custom gridworld matrix (12 x 12)
# Optimal Score = 18
gmatrix = [[  0,   0,   0,   5,   0,   0,   0,   0,   0,   0,   0,   0], 
           [  0,   0,   0,   0,   0,   0,   3,   0,   0,   0,   0,   0],  
           [  0,  10,   0,   0, -20, -20, -20, -20, -20, -20, -20, -20],  
           [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],  
           [  0,   0,   0,   0,   5,   0,   0,   0,   5,   0,   0,   0],  
           [  0,   0,   0, -10, -10, -20, -20, -20,   0,   0,  10,   0],  
           [-20, -20, -20, -10, -10,   0,   0,   0,   0,   0,   0,   0],  
           [  0,   0,   0,   0,   5,   0,   0,   0,   5,   0,   0,   0],  
           [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],  
           [  0,  10,   0,   0, -20, -20, -20, -20, -20, -20, -20, -20],  
           [  0,   0,   0,   0,   0,   0,   3,   0,   0,   0,   0,   0],  
           [  0,   0,   0,   5,   0,   0,   0,   0,   0,   0,   0,   0]]

env = Gridworld(gmatrix=gmatrix, delta_s=1.0, energy=10.0, start=[9, 11], end=[9, 0], fstate=[7, 7])
_max_global_steps = 1000000

In [ ]:
# Custom GRIDWORLD C (MULTIPATH SELECTION TEST)
# Create custom gridworld matrix (12 x 13)
# Optimal Score = 24
gmatrix = [[  0, -10,   0,   0,   0,   0,   0,   0,   0,   3,   0,   0,   0],
           [  0, -10,   0,   0,   5,   0,   0,  10,   0,   0,   0,   0,   0],
           [  0, -10,   0,   5, -10,   0,   3, -10,   0,   5, -20,   5,   0],
           [  0, -10,   0,   0, -10,   0,   0, -10,   0,   0, -20,   0,   0],
           [-20, -20,   0,   0, -20,   0,   0, -20, -20, -20, -20,   3,   0],
           [ 10,   0,   3,   0, -20,   5,   0,   0,   0,   0, -20,   0,   5],
           [  0,   0,   0,   0, -20,   0,   0,   5,   0,   3, -20,   0,   0],
           [  0,   0, -20, -20, -20, -20, -20, -20,   0,   0, -20,   0,   0],
           [  0,   0,   0,   0, -20,   0,  10,   0,   0,   0,   0,   3,   0],
           [  0,   5,   0,   0, -20,   0,   0,   0,   0,   5,   0,   0,   0],
           [  0,   0,   0,   0,   0,   0,   0, -20, -10, -10, -10, -10, -10],
           [  0,   0,   0,   0,   5,   0,   0, -20,   0,   0,   0,   0,   0]]

env = Gridworld(gmatrix=gmatrix, delta_s=1.0, energy=5.0, start=[2, 11], end=[9, 3], fstate=[7, 7])
_max_global_steps = 1000000

In [ ]:
# Custom GRIDWORLD D (GOAL PERSEVERANCE TEST)
# Create custom gridworld matrix (14 x 14)
# Optimal Score = 7
gmatrix = [[  0,   0,   0,   0,   0,   0,   0,   0,   5,   0,   0,   0,   0,   0],
           [  0,  10,   0,   0,   0,   5,   0,   0,   0,   0,   0,   0,   0,   0],
           [  0,   0,   0,   0,   0, -20, -20, -20, -20, -20, -20, -20,   5,   0],
           [  0,   0,   0, -20, -20, -20,   0,   0,   5,   0,   0, -20,   0,   0],
           [  0,   0,   0, -20,   0,   0,   0,   0,   0,   0,   0, -20,   0,   0],
           [  0,   5,   0, -20,   0,   5,   0, -20, -20,   0,   0, -20,   0,   0],
           [  0,   0,   0, -20,   0,   0,   0,   0, -20,   0,   0, -20,   0,   0],
           [  0,   0,   0, -20, -20, -20,   0, -20, -20,   0,   5, -20,   0,   5],
           [  0,   0,   0,   0,   0, -20, -20, -20,   0,   0,   0, -20,   0,   0],
           [  0,   0,   5,   0,   0,   0,   0,   0,   0,   0,   0, -20,   0,   0],
           [-20, -20, -20, -20,   0,   0,   5,   0,   0, -20, -20, -20,   0,   0],
           [  0,   0,   0, -20, -20, -20, -20, -20, -20, -20,   0,   0,   0,   0],
           [  0,   0,   0,   0,   0,   0,   0,   0,   0,   5,   0,   0,  10,   0],
           [  0,   0,   0,   0,   0,   5,   0,   0,   0,   0,   0,   0,   0,   0]]

env = Gridworld(gmatrix=gmatrix, delta_s=1.0, energy=5.0, start=[0, 13], end=[6, 7], fstate=[7, 7])
_max_global_steps = 1500000

In [ ]:
# Custom GRIDWORLD E (ROUTINE DISCOVERY TEST)
# Create custom gridworld matrix (10 x 12)
# Optimal Score = 16
gmatrix = [[  0,   0,   0,   0,   0, -20,   0,   0,   0,   0,   5,  10],
           [  0,   0,   0,   0,   0, -20,   0,   5,   0,   0,   0,   0],
           [  0,   0,   0,   0,   0, -20,   0,   0,   0,   0,   0,   0],
           [  0,   5,   0,   0,   0,  -7,   0,   0, -20,  -7,  -7, -20],
           [  0,   0,   0,   0,   0,  -7,   5,   0, -20,   0,   0,   0],
           [  0,   0,   0,   0,   3,  -7,   0,   5, -20,   0,   0,   0],
           [  0,   0,   0,   0,   0,  -7,   0,   0, -20,   0,   5,   0],
           [  0,   5,   0,  10,   0, -20, -20, -20, -20,   0,   0,   0],
           [  0,   0,   0,   0,   0, -20,   0,   0,   0,   0,   5,   0],
           [  0,   0,   0,   0,   0, -20,   0,  10,   0,   0,   0,   0]]

env = Gridworld(gmatrix=gmatrix, delta_s=1.0, energy=10.0, start=[0, 0], end=[11, 9], fstate=[7, 7])
_max_global_steps = 1000000

In [ ]:
# Custom GRIDWORLD F (TIGHT NAVIGATION TEST)
# Create custom gridworld matrix (13 x 10)
# Optimal Score = 7
gmatrix = [[  0, -10, -20, -20, -20, -20, -20, -20, -20, -20],
           [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
           [  0,   5,   0,   0,   5,   0,   0,   0,   0,   0],
           [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
           [-20, -20, -20, -20, -20, -20, -20, -10,   0,   0],
           [  0,   0,   0,   0,   5,   0,   0,   0,   0,   0],
           [  0,   0,   0,   0,   0,   0,   0,   0,   0,  10],
           [ 10,   0, -20, -20, -20, -20, -20, -20, -20, -20],
           [  0,   0, -20,   0,   0,   0,   0,   0,   0,   0],
           [  0,   0, -20,   0,   0,   5,   0, -10,   0,   0],
           [  0,   0, -20,   0,   0,   0,   0, -20,   5,   0],
           [  0,   5, -10,   0,   5,   0,   0, -20,   0,   0],
           [  0,   0,   0,   0,   0,   0,   0, -20,   0,   0]]

env = Gridworld(gmatrix=gmatrix, delta_s=1.0, energy=5.0, start=[9, 12], end=[0, 0], fstate=[7, 7])
_max_global_steps = 1500000

In [ ]:
# Custom GRIDWORLD G (OPTIMAL POLICY DISCOVERY TEST)
# Create custom gridworld matrix (16 x 16)
# Optimal Observed Machine Score = 77
gmatrix = [[-18,   0,   0,   0,   0,   0, -10,   5,   3,   0,  10,   0,   0,   0,   3,   0],
           [  0,   0,  -9,   3,   0,   0,   0,   0,  -5,   9,  -8,  -4, -19,   0,   0,   0],
           [  6,   5,   7,   1,   0,   4,   0, -17, -11,  -3,   2,   8,   0,   4,   1, -15],
           [-10,   0,  -7,   1, -13,   0,   0,   6,   0, -11, -10,   0,   0,  -7,   0,  -8],
           [  0, -18,   0, -10,  10,  -1,  -9,   0, -16, -18,  -2,   0,   0, -12, -13,   2],
           [ -2,  -2,   8,   0,   0, -10,   0,   0,   0,  -5,   0,   7,   0,   0,   0,   0],
           [ -8, -16,   0,   0,   0,  -9, -13,   5,  -8,   0,   0,   8,   2,   0,   6, -20],
           [  0,   0, -12,   4, -19,   8, -15,  10, -13, -18,   0, -17, -10,   0,   0,   0],
           [-19,   0,   7,   1,   0,  -5,   0,  -9,   0,   5,   3,   0,   0,  -9,  10,   0],
           [ -3,   8,   0,  -5,   0,   0,   5,   0,   0,   0,   0,   0, -13,   0,   0, -12],
           [-14,   0,   0,   0,   9,   0,   0, -15,   0,  -6,   4,   0,   0,   0,  -5, -20],
           [-17,   1,   0,   0,   4, -10,   0,   8, -19,   0,   0,   0,   0,  -4,   9,   0],
           [  0,   7,   0,   0,   0,   0,   4,   0,  -5, -19,   0,   8,   0,  -5, -12,  -8],
           [ -9,   0,  10,   0, -18,   0, -14,   0,   0,  -1,  -5,  -1,  -9,   0, -18, -10],
           [  1,  -3,   0,  -5,   0,   4,  -3,  10,   5,   2,   0,  -5,   0,   6, -12,   0],
           [  0,   0,   0,   4,  -9,   0,  -2,   0, -15, -10,   0,   0, -15,   0,  -2,   0]]

env = Gridworld(gmatrix=gmatrix, delta_s=1.5, energy=10.0, start=[13, 15], end=[2, 0], fstate=[7, 7])
_max_global_steps = 1000000

# Training and Testing

In [17]:
policy_train = Train(parallelism=10, max_global_steps=_max_global_steps, model_dir="a3c_models/env_A/training", custom_env=env)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.dense instead.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Please switch to tf.train.get_global_step


/home/arczer0/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Use the `axis` argument instead


In [18]:
policy_train.run(t_max=8, eval_every=10000, test_epi=30)

INFO:tensorflow:worker_1: local Step 0, global step 0
INFO:tensorflow:worker_2: local Step 0, global step 1
INFO:tensorflow:worker_3: local Step 0, global step 2
INFO:tensorflow:worker_5: local Step 0, global step 3
INFO:tensorflow:worker_4: local Step 0, global step 5
INFO:tensorflow:worker_7: local Step 0, global step 6
INFO:tensorflow:worker_6: local Step 0, global step 7
INFO:tensorflow:worker_10: local Step 0, global step 8
INFO:tensorflow:worker_9: local Step 0, global step 13
INFO:tensorflow:worker_8: local Step 0, global step 25
INFO:tensorflow:worker_5: local Step 50, global step 455
INFO:tensorflow:worker_4: local Step 50, global step 464
INFO:tensorflow:worker_10: local Step 50, global step 470
INFO:tensorflow:worker_8: local Step 50, global step 487
INFO:tensorflow:worker_7: local Step 50, global step 504
INFO:tensorflow:worker_6: local Step 50, global step 506
INFO:tensorflow:worker_2: local Step 50, global step 513
INFO:tensorflow:worker_1: local Step 50, global step 528


INFO:tensorflow:worker_10: local Step 700, global step 6841
INFO:tensorflow:worker_1: local Step 700, global step 6902
INFO:tensorflow:worker_8: local Step 700, global step 6949
INFO:tensorflow:worker_2: local Step 700, global step 6970
INFO:tensorflow:worker_4: local Step 700, global step 6975
INFO:tensorflow:worker_6: local Step 700, global step 6978
INFO:tensorflow:worker_5: local Step 700, global step 6990
INFO:tensorflow:worker_9: local Step 700, global step 7055
INFO:tensorflow:worker_7: local Step 700, global step 7115
INFO:tensorflow:worker_3: local Step 700, global step 7243
INFO:tensorflow:worker_10: local Step 750, global step 7353
INFO:tensorflow:worker_1: local Step 750, global step 7417
INFO:tensorflow:worker_2: local Step 750, global step 7441
INFO:tensorflow:worker_8: local Step 750, global step 7444
INFO:tensorflow:worker_6: local Step 750, global step 7449
INFO:tensorflow:worker_5: local Step 750, global step 7468
INFO:tensorflow:worker_4: local Step 750, global step 

INFO:tensorflow:worker_1: local Step 1200, global step 11840
Episode:  24	 Final Score:  0.00	 Acc. Reward: -1.35	 Min_Dist:  11.00	 Epi_Len:  10.00
Episode:  25	 Final Score:  0.00	 Acc. Reward: -1.35	 Min_Dist:  11.00	 Epi_Len:  10.00
INFO:tensorflow:worker_5: local Step 1200, global step 11945
INFO:tensorflow:worker_4: local Step 1200, global step 11951
INFO:tensorflow:worker_6: local Step 1200, global step 11976
INFO:tensorflow:worker_2: local Step 1200, global step 11986
INFO:tensorflow:worker_8: local Step 1200, global step 11999
Episode:  26	 Final Score:  0.00	 Acc. Reward: -1.35	 Min_Dist:  10.00	 Epi_Len:  10.00
Episode:  27	 Final Score:  0.00	 Acc. Reward: -1.07	 Min_Dist:  10.00	 Epi_Len:   3.00
INFO:tensorflow:worker_9: local Step 1200, global step 12088
Episode:  28	 Final Score:  0.00	 Acc. Reward: -0.80	 Min_Dist:   8.00	 Epi_Len:  13.00
INFO:tensorflow:worker_7: local Step 1200, global step 12200
Episode:  29	 Final Score:  0.00	 Acc. Reward: -1.35	 Min_Dist:   9.00	 

INFO:tensorflow:worker_6: local Step 1800, global step 18010
INFO:tensorflow:worker_2: local Step 1800, global step 18032
INFO:tensorflow:worker_8: local Step 1800, global step 18066
INFO:tensorflow:worker_9: local Step 1800, global step 18134
INFO:tensorflow:worker_10: local Step 1850, global step 18196
INFO:tensorflow:worker_7: local Step 1800, global step 18218
INFO:tensorflow:worker_3: local Step 1800, global step 18289
INFO:tensorflow:worker_1: local Step 1850, global step 18348
INFO:tensorflow:worker_4: local Step 1850, global step 18357
INFO:tensorflow:worker_5: local Step 1850, global step 18408
INFO:tensorflow:worker_6: local Step 1850, global step 18514
INFO:tensorflow:worker_2: local Step 1850, global step 18570
INFO:tensorflow:worker_8: local Step 1850, global step 18582
INFO:tensorflow:worker_9: local Step 1850, global step 18646
INFO:tensorflow:worker_10: local Step 1900, global step 18675
INFO:tensorflow:worker_7: local Step 1850, global step 18692
INFO:tensorflow:worker

INFO:tensorflow:worker_8: local Step 2250, global step 22459
INFO:tensorflow:worker_6: local Step 2250, global step 22523
INFO:tensorflow:worker_2: local Step 2250, global step 22540
INFO:tensorflow:worker_10: local Step 2300, global step 22648
INFO:tensorflow:worker_3: local Step 2250, global step 22682
INFO:tensorflow:worker_9: local Step 2250, global step 22699
INFO:tensorflow:worker_7: local Step 2250, global step 22827
INFO:tensorflow:worker_5: local Step 2300, global step 22853
INFO:tensorflow:worker_4: local Step 2300, global step 22861
INFO:tensorflow:worker_1: local Step 2300, global step 22891
INFO:tensorflow:worker_8: local Step 2300, global step 22998
INFO:tensorflow:worker_6: local Step 2300, global step 23022
INFO:tensorflow:worker_2: local Step 2300, global step 23078
INFO:tensorflow:worker_9: local Step 2300, global step 23187
INFO:tensorflow:worker_10: local Step 2350, global step 23199
INFO:tensorflow:worker_3: local Step 2300, global step 23205
INFO:tensorflow:worker

...

INFO:tensorflow:worker_2: local Step 97900, global step 982263
INFO:tensorflow:worker_8: local Step 98250, global step 982276
INFO:tensorflow:worker_4: local Step 98200, global step 982286
INFO:tensorflow:worker_10: local Step 98350, global step 982308
Episode:  21	 Final Score:  1.00	 Acc. Reward:  1.58	 Min_Dist:   0.00	 Epi_Len:  23.00
INFO:tensorflow:worker_7: local Step 98450, global step 982433
INFO:tensorflow:worker_6: local Step 98300, global step 982453
Episode:  22	 Final Score:  1.00	 Acc. Reward:  1.34	 Min_Dist:   0.00	 Epi_Len:  29.00
INFO:tensorflow:worker_1: local Step 98500, global step 982564
Episode:  23	 Final Score:  0.00	 Acc. Reward: -0.52	 Min_Dist:   2.00	 Epi_Len:  26.00
INFO:tensorflow:worker_3: local Step 98100, global step 982639
INFO:tensorflow:worker_5: local Step 98200, global step 982658
INFO:tensorflow:worker_9: local Step 98200, global step 982683
Episode:  24	 Final Score:  1.00	 Acc. Reward:  1.12	 Min_Dist:   0.00	 Epi_Len:  28.00
INFO:tensorflow:w

INFO:tensorflow:worker_6: local Step 98850, global step 988007
INFO:tensorflow:worker_1: local Step 99050, global step 988074
INFO:tensorflow:worker_9: local Step 98750, global step 988177
INFO:tensorflow:worker_5: local Step 98750, global step 988184
INFO:tensorflow:worker_2: local Step 98500, global step 988215
INFO:tensorflow:worker_8: local Step 98850, global step 988233
INFO:tensorflow:worker_3: local Step 98650, global step 988254
INFO:tensorflow:worker_10: local Step 98950, global step 988275
INFO:tensorflow:worker_7: local Step 99050, global step 988319
INFO:tensorflow:worker_4: local Step 98800, global step 988341
INFO:tensorflow:worker_6: local Step 98900, global step 988489
INFO:tensorflow:worker_1: local Step 99100, global step 988575
INFO:tensorflow:worker_2: local Step 98550, global step 988692
INFO:tensorflow:worker_5: local Step 98800, global step 988705
INFO:tensorflow:worker_8: local Step 98900, global step 988708
INFO:tensorflow:worker_9: local Step 98800, global ste

Episode:  25	 Final Score:  2.00	 Acc. Reward:  1.66	 Min_Dist:   0.00	 Epi_Len:  21.00
INFO:tensorflow:worker_2: local Step 98950, global step 992724
INFO:tensorflow:worker_3: local Step 99100, global step 992772
INFO:tensorflow:worker_10: local Step 99400, global step 992786
INFO:tensorflow:worker_7: local Step 99500, global step 992795
Episode:  26	 Final Score:  1.00	 Acc. Reward:  1.58	 Min_Dist:   0.00	 Epi_Len:  23.00
INFO:tensorflow:worker_4: local Step 99250, global step 992836
Episode:  27	 Final Score:  2.00	 Acc. Reward:  1.66	 Min_Dist:   0.00	 Epi_Len:  21.00
INFO:tensorflow:worker_6: local Step 99350, global step 993012
Episode:  28	 Final Score:  1.00	 Acc. Reward:  1.58	 Min_Dist:   0.00	 Epi_Len:  23.00
INFO:tensorflow:worker_1: local Step 99550, global step 993067
Episode:  29	 Final Score:  1.00	 Acc. Reward:  1.50	 Min_Dist:   0.00	 Epi_Len:  25.00
logfile on global steps=990000.0 saved...
Overall Results: Success Rate: 26/30, Ave. Score:  1.27, Ave. Rewards:  1.20

INFO:tensorflow:worker_10: local Step 100000, global step 998747
INFO:tensorflow:worker_3: local Step 99700, global step 998748
INFO:tensorflow:worker_7: local Step 100100, global step 998811
INFO:tensorflow:worker_4: local Step 99850, global step 998822
INFO:tensorflow:worker_1: local Step 100150, global step 999076
INFO:tensorflow:worker_6: local Step 99950, global step 999088
INFO:tensorflow:worker_8: local Step 99950, global step 999139
INFO:tensorflow:worker_9: local Step 99850, global step 999169
INFO:tensorflow:worker_5: local Step 99850, global step 999186
INFO:tensorflow:worker_2: local Step 99600, global step 999219
INFO:tensorflow:worker_10: local Step 100050, global step 999254
INFO:tensorflow:worker_3: local Step 99750, global step 999278
INFO:tensorflow:worker_4: local Step 99900, global step 999312
INFO:tensorflow:worker_7: local Step 100150, global step 999321
INFO:tensorflow:worker_1: local Step 100200, global step 999566
INFO:tensorflow:worker_6: local Step 100000, gl

Exception in thread Thread-14:
Traceback (most recent call last):
  File "/home/arczer0/anaconda3/envs/tensorflow/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/arczer0/anaconda3/envs/tensorflow/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-14-ab3382bde6d2>", line 129, in <lambda>
    target=lambda: self.pe.continuous_eval(eval_every, test_epi, sess, coord))
  File "<ipython-input-13-625318afd089>", line 169, in continuous_eval
    self.run_test(sess, num_ep, global_t, log=True)
  File "<ipython-input-13-625318afd089>", line 140, in run_test
    stats = self.eval_once(sess, t_epi=ep, render=render)
  File "<ipython-input-13-625318afd089>", line 77, in eval_once
    action_probs = self._policy_net_predict(state, sess)
  File "<ipython-input-13-625318afd089>", line 48, in _policy_net_predict
    preds = sess.run(self.policy_net.predictions, feed_dict)
  File "/home/arczer0/anacon

In [19]:
policy_train.test(num_ep=30)

Loading model checkpoint: a3c_models/env_A/training/model/-100
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from a3c_models/env_A/training/model/-100
Episode:   0	 Final Score:  1.00	 Acc. Reward:  1.58	 Min_Dist:   0.00	 Epi_Len:  23.00
Episode:   1	 Final Score:  1.00	 Acc. Reward:  1.58	 Min_Dist:   0.00	 Epi_Len:  23.00
Episode:   2	 Final Score:  1.00	 Acc. Reward:  1.50	 Min_Dist:   0.00	 Epi_Len:  25.00
Episode:   3	 Final Score:  2.00	 Acc. Reward:  1.66	 Min_Dist:   0.00	 Epi_Len:  21.00
Episode:   4	 Final Score:  1.00	 Acc. Reward:  1.50	 Min_Dist:   0.00	 Epi_Len:  25.00
Episode:   5	 Final Score:  0.00	 Acc. Reward: -1.08	 Min_Dist:   5.00	 Epi_Len:  20.00
Episode:   6	 Final Score:  1.00	 Acc. Reward:  1.34	 Min_Dist:   0.00	 Epi_Len:  29.00
Episode:   7	 Final Score:  2.00	 Acc. Reward:  1.66	 Min_Dist:   0.00	 Epi_Len:  21.00
Episode:   8	 Final Score:  1.00	 Acc. Reward:  1.36	 Min_Dist:   0

# Plotting Log Results

In [20]:
test_log = LogFile('a3c_models/env_A/training/test_log')

In [21]:
data = test_log.get_saved_data('logfile')

In [22]:
# Get global steps
stp = []
for i in range(np.size(data, 0)):
    stp.append(data[i][0])
    
print(stp)
print("Array Length: {}".format(len(stp)))

[10000, 20001, 30003, 40002, 50000, 60002, 70000, 80001, 90000, 100000, 110001, 120000, 130001, 140000, 150000, 160000, 170004, 180000, 190001, 200004, 210000, 220001, 230000, 240000, 250000, 260000, 270000, 280000, 290000, 300000, 310000, 320000, 330000, 340000, 350000, 360000, 370000, 380000, 390000, 400000, 410000, 420000, 430003, 440001, 450000, 460000, 470000, 480000, 490000, 500000, 510000, 520000, 530000, 540000, 550000, 560000, 570002, 580000, 590000, 600000, 610000, 620000, 630000, 640000, 650000, 660000, 670000, 680000, 690000, 700001, 710000, 720000, 730000, 740000, 750000, 760000, 770000, 780000, 790000, 800000, 810000, 820000, 830000, 840001, 850000, 860000, 870000, 880000, 890000, 900000, 910000, 920000, 930001, 940000, 950000, 960000, 970000, 980000, 990000]
Array Length: 99


In [23]:
# Get mean rewards
rwd = []
for i in range(np.size(data, 0)):
    rwd.append(data[i][5])
    
# Get mean score
scr = []
for i in range(np.size(data, 0)):
    scr.append(data[i][4])

# Get mean distance to goal
dst = []
for i in range(np.size(data, 0)):
    dst.append(data[i][6])

In [24]:
import matplotlib.pyplot as plot

In [25]:
_range = 99 # Length of result to be plotted
_scale = 10.0 # ( max_steps / _range)
x_step = 50
x_stop = 1000

In [27]:
# Plot rewards

# Define plot size
plot.figure(num=None, figsize=(10,6))

x_axis = []
for i in range(_range):
    x_axis.append(i * _scale)

plot.plot(x_axis, rwd[0:_range], color='teal')

# Title
plot.title('Total Rewards')

# Labels
plot.xlabel('steps (1x1000)')
plot.ylabel('ave. total rewards')
plot.xticks(np.arange(start=0.0, stop=x_stop, step=x_step))
plot.yticks(np.arange(start=-2.0, stop=2.2, step=0.2))

# Add grid
plot.grid(False)

# show plot
#plot.show()

# save plot
plot.savefig('a3c_models/plots/gworldA_c_rewards.jpg')
plot.close()

In [28]:
# Plot distance to goal

# Define plot size
plot.figure(num=None, figsize=(10,6))

x_axis = []
for i in range(_range):
    x_axis.append(i * _scale)

plot.plot(x_axis, dst[0:_range], color='green')

# Title
plot.title('Distance To Goal')

# Labels
plot.xlabel('steps (1x1000)')
plot.ylabel('ave. distance to endpoint')
plot.xticks(np.arange(start=0.0, stop=x_stop, step=x_step))
plot.yticks(np.arange(start=0.0, stop=12.0, step=1.0))

# Add grid
plot.grid(False)

# show plot
#plot.show()

# save plot
plot.savefig('a3c_models/plots/gworldA_c_distance.jpg')
plot.close()

In [29]:
# Plot score

# Define plot size
plot.figure(num=None, figsize=(10,6))

x_axis = []
for i in range(_range):
    x_axis.append(i * _scale)

plot.plot(x_axis, scr[0:_range], color='orange')

# Title
plot.title('Average Score')

# Labels
plot.xlabel('steps (1x1000)')
plot.ylabel('average score')
plot.xticks(np.arange(start=0.0, stop=x_stop, step=x_step))
plot.yticks(np.arange(start=0.0, stop=2.2, step=0.2))

# Add grid
plot.grid(False)

# show plot
#plot.show()

# save plot
plot.savefig('a3c_models/plots/gworldA_c_score.jpg')
plot.close()